KELOMPOK 103132430001:
- Nama: Immanuel Raditya Deo Pratama Pambudhi 

- NIM: 103132430001

# FUZZIFIKASI
Konsep Matematika :

Garis Turun: (batas atas - x) / (batas atas - batas bawah)

Garis Naik: (x - batas bawah) / (batas atas - batas bawah)

In [17]:
# Membuat garis linear untuk derajat keanggotaan fuzzy dengan analogi bentuk segitiga dan puncak
def garis_turun(x, a, b):
    # Kamus:
    # a = titik mulai turun (nilai 1)
    # b = titik akhir turun (nilai 0)
    if x <= a:
        return 1.0
    elif a < x < b:
        return (b - x) / (b - a)
    else:
        return 0.0

def garis_naik(x, a, b):
    # Kamus:
    # a = titik mulai naik (nilai 0)
    # b = titik puncak naik (nilai 1)
    if x <= a:
        return 0.0
    elif a < x < b:
        return (x - a) / (b - a)
    else:
        return 1.0

# Kamus: 
# a = batas kiri (0)
# b = puncak (1)
# c = batas kanan (0)
def garis_segitiga(x, a, b, c):
    if a < x <= b:
        return (x - a) / (b - a) # Sisi Naik
    elif b < x < c:
        return (c - x) / (c - b) # Sisi Turun
    else:
        return 0.0

# Fuzzifikasi nilai rekomendasi
# Kita pakai 3 level dengan skala rekomendasi (1-100): Buruk(1-50), Cukup(40-80), Bagus(70-100)
def fuzifikasi_servis(nilai):
    hasil = {}
    hasil['buruk'] = garis_turun(nilai, 1, 50)
    hasil['cukup'] = garis_segitiga(nilai, 40, 60, 80)
    hasil['bagus'] = garis_naik(nilai, 70, 100)
    
    return hasil

# Fuzzifikasi harga
# Kita pakai 3 level dengan skala harga (1-10): Murah(1-5), Sedang(4-9), Mahal(8-10)
def fuzifikasi_harga(nilai):
    hasil = {}
    hasil['murah'] = garis_turun(nilai, 1, 5)
    hasil['sedang'] = garis_segitiga(nilai, 4, 6.5, 9)
    hasil['mahal'] = garis_naik(nilai, 8, 10)
    
    return hasil

# TAHAP INFERENSI
Desain Aturan (Rule Base)Kita punya 2 input dengan masing-masing 3 kategori.

- Servis: Buruk, Cukup, Bagus

- Harga: Murah, Sedang, Mahal 

Konstanta ouput(Sugeno) kita akan pakai 2 nilai tegas, yaitu:

- TIDAK LAYAK = 20 (Skor rendah)

- LAYAK = 100 (Skor tinggi)

In [18]:
# Mendefinisikan dan membuat otak dari sistem menggunakan metode Sugeno 0-Order
def inferensi(servis, harga):
    TIDAK_LAYAK = 20
    LAYAK = 100
    rules = []

    # Pembuatan aturan fuzzy
    alpha1 = min(servis['buruk'], harga['murah'])
    rules.append([alpha1, TIDAK_LAYAK])

    alpha2 = min(servis['buruk'], harga['sedang'])
    rules.append([alpha2, TIDAK_LAYAK])

    alpha3 = min(servis['buruk'], harga['mahal'])
    rules.append([alpha3, TIDAK_LAYAK])

    alpha4 = min(servis['cukup'], harga['murah'])
    rules.append([alpha4, LAYAK])

    alpha5 = min(servis['cukup'], harga['sedang'])
    rules.append([alpha5, TIDAK_LAYAK])

    alpha6 = min(servis['cukup'], harga['mahal'])
    rules.append([alpha6, TIDAK_LAYAK])

    alpha7 = min(servis['bagus'], harga['murah'])
    rules.append([alpha7, LAYAK])

    alpha8 = min(servis['bagus'], harga['sedang'])
    rules.append([alpha8, LAYAK])

    alpha9 = min(servis['bagus'], harga['mahal'])
    rules.append([alpha9, LAYAK])
    
    return rules

# DEFUZIFIKASI
Untuk mengubah kembali menjadi nilai nyata dalam angka menggunakan rumus Rata-rata Tertimbang (Weighted Average) karena metode yang dipakai Sugeno:

Skor Akhir = (Total (Alpha) x Nilai Z) / (Total Alpha)

Artinya: Nilai penentu rekomendasi nantinya yang akan diambil adalah yang paling dominan

In [19]:
def defuzzifikasi(list_rules):
    pembilang = 0.0
    penyebut = 0.0  
    
    for rule in list_rules:
        alpha = rule[0]
        z = rule[1]
        
        pembilang += (alpha * z)
        penyebut += alpha
        
    if penyebut == 0:
        return 0.0
    
    return pembilang / penyebut

# PROGRAM UTAMA

In [20]:
# Pembuatan fungsi baca dataset
def baca_file(nama_file):
    data_bengkel = []

    with open(nama_file, 'r') as file:
        baris_data = file.readlines()

        for i in range(1, len(baris_data)):
            baris = baris_data[i].strip() 
            if not baris: 
                continue
                
            kolom = baris.split(',') 
            bengkel = {
                'id': kolom[0],
                'servis': float(kolom[1]),
                'harga': float(kolom[2])
            }
            data_bengkel.append(bengkel)
    return data_bengkel

# Inisialisasi variabel untuk dataset dan array hasil analisis
data_bengkel = baca_file('bengkel.csv')
hasil_analisis = []

# Implementasi proses fuzzy logic
for bengkel in data_bengkel:
    servis = fuzifikasi_servis(bengkel['servis'])
    harga = fuzifikasi_harga(bengkel['harga'])
    rules = inferensi(servis, harga)
    skorAkhir = defuzzifikasi(rules)

    hasil_analisis.append({
        'id': bengkel['id'],
        'servis': bengkel['servis'],
        'harga': bengkel['harga'],
        'skor': skorAkhir
    })

# Sorting data secara menurun berdasarkan skor & mengambil 5 sampel terbaik
hasil_analisis.sort(key=lambda x: x['skor'], reverse=True)
top_5 = hasil_analisis[:5]
print("\n5 BENGKEL TERBAIK:")
print(f"\n{'ID':<5} | {'Servis':<8} | {'Harga':<6} | {'Skor':<10}")
for b in top_5:
    print(f"{b['id']:<5} | {b['servis']:<8} | {b['harga']:<6} | {b['skor']:.2f}")

# Simpan kesimpulan di dalam file dalam format .xls
nama_file_output = 'Rekomendasi_Bengkel.xls' 
with open(nama_file_output, 'w') as file:
    file.write("ID\tServis\tHarga\tSkor\n") 
    for b in top_5:
        file.write(f"{b['id']}\t{b['servis']}\t{b['harga']}\t{b['skor']}\n")

print(f"\nKesimpulan bengkel terbaik disimpan di file: {nama_file_output}")


5 BENGKEL TERBAIK:

ID    | Servis   | Harga  | Skor      
2     | 93.0     | 9.0    | 100.00
4     | 72.0     | 1.0    | 100.00
7     | 83.0     | 9.0    | 100.00
8     | 87.0     | 8.0    | 100.00
9     | 75.0     | 1.0    | 100.00

Kesimpulan bengkel terbaik disimpan di file: Rekomendasi_Bengkel.xls
